# Benchmark Power Flow

This notebook aims to beckmark the power flow

In [3]:
import warnings
import logging
import numpy as np
import pandas as pd

import andes
import ams
import ams.benchmarks as bp

# Configure AMS logger
andes.config_logger(stream_level=20)

# Display tool versions
_ = bp.get_tool_versions()

ams.config_logger(stream_level=50)

Last run time: 2024-11-24 11:33:34
Python: 3.10.0 | packaged by conda-forge | (default, Nov 20 2021, 02:27:15) [Clang 11.1.0 ]

Tool        Version                
----------  -----------------------
ltbams      0.9.12                 
andes       1.9.1.post100+g2b17faac
cvxpy       1.5.3                  
gurobipy    11.0.3                 
mosek       10.2.6                 
piqp        0.4.2                  
pandapower  2.14.11                
numba       0.60.0                 


In [2]:
%matplotlib inline

In [3]:
cases = [
    './cases/case14.m',
    './cases/case39.m',
    './cases/case89pegase.m',
    './cases/case118.m',
    './cases/npcc.m',
    './cases/wecc.m',
    './cases/case300.m',
    './cases/pglib_opf_case1354_pegase.m',
    './cases/pglib_opf_case2869_pegase.m',
    './cases/pglib_opf_case4020_goc.m',
    './cases/pglib_opf_case5658_epigrids.m',
    './cases/pglib_opf_case7336_epigrids.m',
]

In [5]:
# Suppress warnings
warnings.filterwarnings('ignore')
# show only errors
logging.getLogger('pandapower').setLevel(logging.ERROR)

# Initialize data structures for storing results
n_iters = 10
n_cases = len(cases)

solvers=['GUROBI', 'MOSEK', 'PIQP', 'pandapower']

time_data = np.zeros((n_iters, n_cases, len(bp.cols_pre) + len(solvers)))
obj_data = np.zeros((n_cases, len(solvers)))

In [6]:
# Run the benchmark for each case and iteration
for n_case, case in enumerate(cases):
    print(f'Case: {case}')
    system = ams.load(case, setup=True, default_config=True, no_output=True)
    if system.Bus.n > 4000:
        system.Line.set(src='rate_a', attr='v', idx=system.Line.idx.v, value=99999)
    for n_iter in range(n_iters):
        pre_time, sol = bp.time_routine(system, routine='DCOPF',
                                        solvers=solvers,
                                        ignore_dpp=True)
        time_data[n_iter, n_case, :len(pre_time)] = np.array([i for i in pre_time.values()])
        time_data[n_iter, n_case, len(pre_time):] = np.array([sol[solver]['time'] for solver in solvers])
    obj = np.array([sol[solver]['obj'] for solver in solvers])
    obj_data[n_case, :] = obj

# Optionally, re-enable warnings if needed elsewhere in your code
warnings.filterwarnings('default')

Case: ./cases/case14.m
Case: ./cases/case39.m
Case: ./cases/case89pegase.m
Case: ./cases/case118.m
Case: ./cases/npcc.m
Case: ./cases/wecc.m
Case: ./cases/case300.m
Case: ./cases/pglib_opf_case1354_pegase.m
Case: ./cases/pglib_opf_case2869_pegase.m
Case: ./cases/pglib_opf_case4020_goc.m
Case: ./cases/pglib_opf_case5658_epigrids.m
Case: ./cases/pglib_opf_case7336_epigrids.m


In [7]:
# Process and display the results
case_names = [case.split('/')[-1].split('.')[0] for case in cases]

obj_dcopf = pd.DataFrame(obj_data, columns=solvers, index=case_names)

time_dcopf = pd.DataFrame(columns=bp.cols_pre+solvers, index=case_names)
for case in case_names:
    time_dcopf.loc[case] = time_data[:, case_names.index(case), :].mean(axis=0)

time_dcopf.iloc[:, :] *= 1000  # Convert time to milliseconds

In [8]:
time_dcopf

,ams_mats,ams_parse,ams_eval,ams_final,ams_postinit,GUROBI,MOSEK,PIQP,pandapower
case14,0.67,1.16,2.24,0.09,0.01,5.35,5.74,4.02,14.08
case39,0.71,1.14,2.2,0.09,0.01,5.75,6.06,4.08,14.2
case89pegase,1.34,1.16,2.33,0.09,0.01,8.27,8.22,10.24,20.01
case118,1.37,1.17,5.45,0.09,0.01,8.89,9.87,4.97,27.7
npcc,1.46,1.14,4.86,0.09,0.01,9.59,10.68,5.24,65.15
wecc,1.58,1.17,3.36,0.09,0.01,9.11,9.05,5.34,39.5
case300,2.23,1.17,7.69,0.09,0.01,14.67,15.71,6.61,37.63
pglib_opf_case1354_pegase,9.37,1.2,67.06,0.22,0.01,61.6,41.66,17.77,524.45
pglib_opf_case2869_pegase,20.69,1.22,244.44,0.35,0.01,130.9,100.78,39.44,1462.82
pglib_opf_case4020_goc,26.91,1.2,125.34,0.63,0.01,279.04,180.08,169.86,1886.75


In [9]:
obj_dcopf.round(2)

,GUROBI,MOSEK,PIQP,pandapower
case14,7642.59,7642.59,7642.59,7642.59
case39,41263.94,41263.94,41263.94,41263.94
case89pegase,5733.37,5733.37,5733.37,5733.37
case118,125947.88,125947.88,125947.88,125947.88
npcc,810033.37,810033.37,810033.37,810016.06
wecc,411706.13,411706.13,411706.13,411706.13
case300,706292.32,706292.32,706292.32,706292.32
pglib_opf_case1354_pegase,1218096.86,1218096.86,1218096.86,1218096.86
pglib_opf_case2869_pegase,2386235.33,2386235.32,2386235.33,2386235.33
pglib_opf_case4020_goc,793634.11,793634.11,793634.11,793634.11


In [10]:
# export results to csv
obj_dcopf.to_csv('../results/results_obj.csv')
time_dcopf.to_csv('../results/results_time.csv')